In [ ]:
#!f2py --f90exec=mpif90 -I. -c -m ocean ocean.f90

In [ ]:
import time
import numpy as np

import wave as ocean

%matplotlib inline

# Definition of model variables

### Model domain / grid parameters

In [ ]:
file1='../data/gbr_south.csv'
file2='../data/topoGBR1000.csv'

# Bathymetric filename
bfile = file1

# Resolution factor
rfac = 4

### Definition of wave parameters

In [ ]:
# Wave heights (m)
H0 = 2

# Define wave source direction at boundary 
# (angle in degrees counterclock wise from horizontal axis)
dir = 300

# Maximum depth for wave influence (m)
wbase = 20

# Sea level position (m)
slvl = 0.

### Definition of sediment parameters

In [ ]:
# Mean grain size diameter in m
d50 = 0.0001

# Steps used to perform sediment transport
tsteps = 1000

# Steps used to perform sediment diffusion
dsteps = 1000

# Model initialisation function

### Loading the bathymetry file

In [ ]:
#help(ocean.wave.__init__)

In [ ]:
wavesed = ocean.wave(filename=bfile,wavebase=wbase,resfac=rfac,dia=d50)

### Defining wave region computational grid

In [ ]:
t0 = time.clock()
wavesed.findland(slvl)
print 'Wave region computation took (s):',time.clock()-t0

### Initialising wave boundary conditions 

In [ ]:
wdir = wavesed.wavesource(dir)

# Waves evolution

The waves are then transformed from deep to shallow water assuming shore-parallel depth contours. The orientation of wave fronts is determine by wave celerity and refraction due to depth variations. 

Travel time in the domain is calculated from Huygen's principle (using an order $\sqrt{5}$ approximation).

Assuming no refraction or loss of energy due to bottom friction, wave power P is conserved from deep to shallow water. 

In [ ]:
#help(wavesed.cmptwaves)

In [ ]:
t0 = time.clock()
wavesed.cmptwaves(src=wdir, h0=H0, sigma=1.)
print 'Wave parameters computation took (s): ',time.clock()-t0

## Visualisation of wave characteristics

In [ ]:
#help(wavesed.plotData)

In [ ]:
size = (15,30)
i1 = 0 
i2 = -1
j1 = 0
j2 = -1

# Zooming to a specific region
# i1 = 170 
# i2 = 260 
# j1 = 0
# j2 = 70 


wavesed.plotData(data='bathy', figsize=size, vmin=0, vmax=0, 
                 fontsize=10, imin=i1, imax=i2, jmin=j1, jmax=j2)

wavesed.plotData(data='travel', figsize=size, tstep=400, vmin=0, vmax=0, 
                 fontsize=10, imin=i1, imax=i2, jmin=j1, jmax=j2)

wavesed.plotData(data='wcelerity', figsize=size, vmin=0, vmax=15, 
                 fontsize=10, stream=3, imin=i1, imax=i2, jmin=j1, jmax=j2)

wavesed.plotData(data='ubot', figsize=size, vmin=0, vmax=2, 
                 fontsize=10, imin=i1, imax=i2, jmin=j1, jmax=j2)

wavesed.plotData(data='shear', figsize=size, vmin=-0.5, vmax=0.5, 
                  fontsize=10, imin=i1, imax=i2, jmin=j1, jmax=j2)

# Sediment entrainment, transport & deposition

Sediment entrainment relates to wave induced shear stress. The transport is computed according to both
wave direction and longshore transport. Deposition is dependent of shear stress and diffusion.

In [ ]:
#help(wavesed.cmptsed)

In [ ]:
t0 = time.clock()
wavesed.cmptsed(sigma=1.,tsteps=tsteps,dsteps=dsteps)
print 'Sediment erosion/deposition computation took (s): ',time.clock()-t0

## Visualisation of sediment transport characteristics

In [ ]:
#help(wavesed.plotData)

In [ ]:
size = (10,20)
# i1 = 0 
# i2 = -1
# j1 = 0
# j2 = -1

# Zooming to a specific region
i1 = 600 
i2 = 1200 
j1 = 0
j2 = 500 

wavesed.plotData(data='fbathy', figsize=size, vmin=0, vmax=0, 
                 fontsize=10, stream=0, imin=i1, imax=i2, jmin=j1, jmax=j2)

wavesed.plotData(data='erodep', figsize=size, vmin=-2., vmax=2., 
                 fontsize=10, stream=0, imin=i1, imax=i2, jmin=j1, jmax=j2)

# Saving wave/sedimentation data

In [ ]:
#waveparams.outputCSV(filename='erodep.csv', seddata=1)